In [67]:
import os
import cv2
import json
import tqdm
import shutil
import pandas as pd

root_path = r"E:\aihub_safety_dataset\\"
label_path = root_path+r"Training_unziped\label_factory_org\\"
label_mod_path = root_path+r"Training_unziped\label_factory_mod\\"
img_path = root_path+r"Training_unziped\image_factory\\"
training_dataset_path = root_path+r"Training_unziped\training_dataset\\"
validation_dataset_path = root_path+r"Training_unziped\validation_dataset\\"
label_file_list = os.listdir(label_path)

1. 이미지와 바운딩 박스 확인

In [70]:
class_number_filter = ["01", "02", "03", "04", "05", "06", "07", "08"]
for idx, label in enumerate(label_file_list):
    if idx < 50:
        json_str = open(label_path + label, "r").readlines()
        json_str = "".join(json_str)
        json_data = json.loads(json_str)
        bbox_infos = [x for x in json_data["annotations"] if x["class"] in class_number_filter]
        if len(bbox_infos) > 0:
            img = cv2.imread(img_path + json_data["image"]["filename"])
            for bbox in bbox_infos:
                if "box" in bbox:
                    cv2.rectangle(img, bbox["box"][:2], bbox["box"][2:4], (0, 255, 0), 2)
                    cv2.putText(img, bbox["class"], bbox["box"][:2], cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0))
            cv2.imshow("img", img)
            cv2.waitKey()
            cv2.destroyAllWindows()

2. 안전장구가 있는 이미지만 필터링 하고 라벨링 txt 파일 생성

In [ ]:
label_made = []
label_not_made = []
class_number_filter = ["01", "02", "04", "05", "06", "07", "08"] #03은 폴리곤으로 되어 있어 03은 제거함
class_number_name = ["belt", "no belt", "gori not fasten", "footgear", "no footgear", "helmet", "no helmet"]
class_info = [] 
for idx, class_number in enumerate(class_number_filter):
    class_info.append({"class_no" : str(idx), "class_org_no" : class_number, "class_name" : class_number_name[idx], "image_cnt" : 0})

for idx, label in enumerate(label_file_list):
    print("{} : 변환중...({:.2f}%)   ".format(label, ((idx+1)/len(label_file_list))*100), end="\r")
    json_str = open(label_path + label, "r", encoding="utf-8").readlines()
    json_str = "".join(json_str)
    json_data = json.loads(json_str)

    w = json_data["image"]["resolution"][0]
    h = json_data["image"]["resolution"][1]
    bbox_infos = [x for x in json_data["annotations"] if x["class"] in class_number_filter]
    if len(bbox_infos) > 0:
        label_str = ""
        for bbox in bbox_infos:
            if "box" in bbox:
                bb_center_x = bbox["box"][0] + ((bbox["box"][2] - bbox["box"][0])/2)
                bb_center_y = bbox["box"][1] + ((bbox["box"][3] - bbox["box"][1])/2)
                bb_w = bbox["box"][2] - bbox["box"][0]
                bb_h = bbox["box"][3] - bbox["box"][1]
                bb_w_scaled = bb_w/w
                bb_h_scaled = bb_h/h
                bb_center_x_scaled = bb_center_x/w
                bb_center_y_scaled = bb_center_y/h
                new_class_no = [x["class_no"] for x in class_info if x["class_org_no"] == bbox["class"]][0]
                class_info_idx = [idx for idx, x in enumerate(class_info) if x["class_org_no"] == bbox["class"]][0]
                label_str += "{}\t{}\t{}\t{}\t{}\n".format(new_class_no, bb_center_x_scaled, bb_center_y_scaled, bb_w_scaled, bb_h_scaled)
                class_info[class_info_idx]["image_cnt"] = class_info[class_info_idx]["image_cnt"] + 1
        if label_str != "":
            label_made.append(json_data["image"]["filename"])
            label_str = label_str[:-2]
            open(label_mod_path + json_data["image"]["filename"].split(".")[0] + ".txt", "w").write(label_str)
        else:
            label_not_made.append(json_data["image"]["filename"])
    else:
        label_not_made.append(json_data["image"]["filename"])

df_class_info = pd.DataFrame(class_info)
df_label_made = pd.DataFrame({ "image_name" : label_made})

df_class_info.to_excel("df_class_info.xlsx", index=False)
df_label_made.to_excel("df_label_made.xlsx", index=False)
df_class_info, df_label_made

4. 안전장구가 없는 이미지는 삭제

In [ ]:
# img_file_list = os.listdir(img_path)
# del_cnt = 0
# for img in img_file_list:
#     if img not in label_made:
#         this_img_path = img_path + img
#         os.remove(this_img_path)

5. 데이터셋 분할(test, validataion)

In [ ]:
df_class_info = pd.read_excel("df_class_info.xlsx")
df_label_made = pd.read_excel("df_label_made.xlsx")

In [39]:
df_class_info

,class_no,class_org_no,class_name,image_cnt
0,0,1,belt,38328
1,1,2,no belt,41541
2,2,4,gori not fasten,8867
3,3,5,footgear,48142
4,4,6,no footgear,456
5,5,7,helmet,65416
6,6,8,no helmet,1579


In [58]:
#전체 데이터 4만여장 중 1만장만 선정
#gori not fasten(2번, 8867개), no footgear(4번, 456개), no helmet(6번, 1579개)은 갯수가 적으므로 적절히 섞어 줌

sample_cnt = 4000
df_for_sample = df_label_made.sample(frac=1, random_state=1234) #이미지를 랜덤하게 섞어 줌
df_for_sample.reset_index(drop=True, inplace=True)
samples = []
gori_not_fasten_cnt = 0
for i in tqdm.trange(len(df_for_sample)):
    this_file_name = df_for_sample.iloc[i, 0]
    this_label = open("{}.txt".format(label_mod_path + this_file_name.split(".")[0]), "r").readlines()
    this_label_list = [x.split("\t")[0] for x in this_label]
    if "4" in this_label_list or "6" in this_label_list: #no footgear와 no helmet이 있는 이미지는 모두 선택
        if this_file_name not in samples:
            samples.append(this_file_name)
    if "2" in this_label_list: #gori not fasten 이미지는 3천장 정도 넣어줌
        gori_not_fasten_cnt += 1
        if gori_not_fasten_cnt <= 3000:
            if this_file_name not in samples:
                samples.append(this_file_name)

print("선별된 이미지 개수 : {}개".format(len(samples)))

100%|██████████| 42292/42292 [00:06<00:00, 6311.80it/s]

선별된 이미지 개수 : 4759개


In [59]:
#1만장에서 남은 갯수만큼 뽑아서 넣어줌
for i in tqdm.trange(len(df_for_sample)):
    this_file_name = df_for_sample.iloc[i, 0]
    if this_file_name not in samples:
        samples.append(this_file_name)
    if len(samples) == sample_cnt:
        break

df_samples = pd.DataFrame({ "image_name" : samples})
print("최종 선별된 이미지 개수 : {}개".format(len(samples)))

 16%|█▌        | 6717/42292 [00:01<00:09, 3682.64it/s]

최종 선별된 이미지 개수 : 10000개


In [64]:
train = df_samples.sample(frac=0.8, random_state=8699)
test = df_samples.drop(train.index)
train

,image_name
8448,S2-N1105M01782.jpg
4579,S2-N1105M26218.jpg
23,S2-N1105M03608.jpg
2611,S2-N1105M17813.jpg
3631,S2-O1103M05839.jpg
...,...
8153,S2-N1105M02012.jpg
2537,S2-N1106M29212.jpg
9655,S2-N1106M14826.jpg
6496,S2-N1106M16112.jpg


In [66]:
print("트레이닝 데이터셋 복사중...")
for i in tqdm.trange(len(train)):
    this_file_name = train.iloc[i]["image_name"].split(".")[0]
    src = cv2.imread("{}{}.jpg".format(img_path, this_file_name))
    resized = cv2.resize(src, (640, 640), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite("{}/images/{}.jpg".format(training_dataset_path, this_file_name), resized)
    shutil.copyfile("{}{}.txt".format(label_mod_path, this_file_name), "{}/labels/{}.txt".format(training_dataset_path, this_file_name))

print("검증 데이터셋 복사중...")
for i in tqdm.trange(len(test)):
    this_file_name = test.iloc[i]["image_name"].split(".")[0]
    src = cv2.imread("{}{}.jpg".format(img_path, this_file_name))
    resized = cv2.resize(src, (640, 640), interpolation=cv2.INTER_LINEAR)
    cv2.imwrite("{}/images/{}.jpg".format(validation_dataset_path, this_file_name), resized)
    shutil.copyfile("{}{}.txt".format(label_mod_path, this_file_name), "{}/labels/{}.txt".format(validation_dataset_path, this_file_name))

트레이닝 데이터셋 복사중...


100%|██████████| 8000/8000 [11:00<00:00, 12.12it/s]


검증 데이터셋 복사중...


100%|██████████| 2000/2000 [02:48<00:00, 11.85it/s]


아래 명령어로 트레이닝 실행!

python C:\Users\poscohrd\Desktop\yolo_transfer_learning\yolov5-master\train.py --img 640 --batch 16 --epochs 30 --data C:\Users\poscohrd\Desktop\yolo_transfer_learning\yolov5-master\dataset.yaml --weights yolov5m.pt --device 0,1

*배치 사이즈가 너무 클 경우 GPU 메모리 에러 발생함 주의할 것

학습이 끝나면 아래와 같이 tensorflow 모델로 변환(필요시)

python yolov5-master\export.py --weights yolov5-master\runs\train\exp11\weights\best.pt --include pb